In [1]:
import sys
sys.path.append('/Users/spacecoffin/Development')

import GravelKicker as gk
import librosa
import numpy as np
import os
import pandas as pd

from datetime import datetime
from supriya.tools import nonrealtimetools

In [2]:
this_dir = '/Users/spacecoffin/Development/GravelKicker/__gen_files'

In [86]:
pmtx = gk.generator.gendy1.gen_params(rows=10)

In [27]:
df = gk.generator.gendy1.format_params(pmtx)
#df = df.set_index(['hash']).sort_index()
df.head()

,adparam,ampscale,ddparam,durscale,ampdist,durdist,knum,minfrequency,maxfrequency,init_cps,hash
0,0.305046,0.738415,0.887657,0.877431,2.0,5.0,14.0,2793.825851,4434.922096,16.0,376fe4fe7ff3ba2bbedc5a1be02fd89a
1,0.683504,0.967584,0.186622,0.620781,5.0,3.0,14.0,440.000000,1864.655046,16.0,bc31fd5649832828c5caa48c2d2f46d6
2,0.417066,0.609886,0.356372,0.071453,5.0,0.0,15.0,14.567618,123.470825,16.0,511bf62ca08eedb54ad577087befc475
3,0.071547,0.552848,0.838256,0.765594,4.0,1.0,11.0,391.995436,830.609395,16.0,90407d20cc24df0f497d742e44bef067
4,0.650456,0.124529,0.715300,0.197939,1.0,3.0,12.0,19.445436,23.124651,16.0,7996829f7f41c02f719edbeff278b880


In [ ]:
%%time

for i, row in df.iterrows():
    
    session = nonrealtimetools.Session()
    
    builder = gk.generator.gendy1.make_builder(row)
    
    out = gk.generator.gendy1.build_out(builder)
    
    synthdef = builder.build()
    
    with session.at(0):
        synth_a = session.add_synth(duration=10, synthdef=synthdef)
    
    gk.util.render_session(session, this_dir, row["hash"])

In [64]:
for i, row in df.iterrows():
    
    y, sr = librosa.load(os.path.join(this_dir, "aif_files", row["hash"] + ".aiff"))
    
    _y_normed = librosa.util.normalize(y)
    _mfcc = librosa.feature.mfcc(y=_y_normed, sr=sr, n_mfcc=13)
    _cent = np.mean(librosa.feature.spectral_centroid(y=_y_normed, sr=sr))
    
    _mfcc_mean = gk.feature_extraction.get_stats(_mfcc)["mean"]
    
    X_row = np.append(_mfcc_mean, _cent)
    
    if i==0:
        X_mtx = X_row
    else:
        X_mtx = np.vstack((X_mtx, X_row))
    

In [72]:
def col_rename_4_mfcc(c):
    if (c < 13):
        return "mfcc_mean_{}".format(c)
    else:
        return "spectral_centroid"

In [77]:
X_df = pd.DataFrame(X_mtx)
X_df = X_df.rename_axis(lambda c: col_rename_4_mfcc(c), axis=1)
#X_df = X_df.rename_axis(lambda c: "mfcc_mean_{}".format(c) if (c < 13) else "spectral_centroid", axis=1)
X_df

,mfcc_mean_0,mfcc_mean_1,mfcc_mean_2,mfcc_mean_3,mfcc_mean_4,mfcc_mean_5,mfcc_mean_6,mfcc_mean_7,mfcc_mean_8,mfcc_mean_9,mfcc_mean_10,mfcc_mean_11,mfcc_mean_12,spectral_centroid
0,61.715024,-9.082724,4.732451,5.725778,0.172767,-0.867081,-0.828309,5.009954,-6.656785,4.234751,1.369022,-2.945906,1.506525,5711.714305
1,108.889868,53.696462,-39.245351,23.945439,-14.006032,8.868904,-7.137677,9.100659,3.106007,16.876341,7.929819,9.733003,-5.253294,3031.093617
2,-118.590227,210.233522,22.371572,36.239547,5.330239,5.733142,-0.662832,9.009006,4.035069,0.750399,-1.876672,4.176212,0.814904,457.125421
3,76.247343,87.439446,-62.293672,21.862616,-6.903217,-0.098767,-2.141063,-2.715470,-5.284541,2.387879,5.325683,20.227300,20.152042,2287.696974
4,-293.055971,186.990610,48.872884,37.750443,36.856604,21.602389,21.831231,16.698051,13.951953,13.992714,11.181573,11.345146,10.005301,150.613536
5,19.029584,168.055112,-23.622233,-7.699141,-15.960146,22.231260,-5.515838,-9.036671,2.470164,8.567555,-14.195794,8.973479,-3.640050,1226.663660
6,-218.161182,200.679395,33.571385,39.642313,30.800382,22.366552,21.019134,14.717270,14.484972,11.074764,10.974083,10.234368,9.223065,258.308694
7,41.896704,-7.734263,11.321656,24.725562,12.592033,1.730682,-3.076871,19.100902,-1.589714,-4.696922,8.778451,5.811193,-8.397733,5336.219216
8,-409.930025,106.165585,76.230345,47.466881,30.600985,24.278361,21.606135,18.531092,15.404551,13.347125,12.274377,11.359286,10.353908,28.753456
9,25.885220,99.638946,-61.426574,1.094605,7.290967,-2.040843,-12.865594,14.930138,-18.243812,9.264674,-6.119540,-0.247084,-1.878331,1997.804326


In [80]:
%%time

pd.DataFrame(feat_row).T

CPU times: user 542 µs, sys: 44 µs, total: 586 µs
Wall time: 554 µs


,0,1,2,3,4,5,6,7,8,9,...,95,96,97,98,99,100,101,102,103,104
0,28.753456,0.466576,-0.028415,-0.088772,-0.036559,-0.021464,-0.020345,-0.010762,-0.007487,0.01248,...,17.260548,11.760914,3.17281,2.103167,1.758607,2.25002,2.184831,2.779369,1.380804,0.854717
